### Use case notebook: schedule CCB team meetings ###

In [8]:
import os
import glob
import numpy as np

# There is a warning in the timeboard library that we want to suppress here
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
from pathlib import Path
import time
import timeboard as tb
import timeboard.calendars.US as US
import datetime
import holidays

# Appearance of the Notebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Import the package
%load_ext autoreload
%autoreload 2
import cadence
from cadence.utils import FileOP
from cadence.mscheduler import Meetings
print(f'Package version: {cadence.__version__}')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Package version: 0.0.2.post1.dev9+g55f2059.d20240827


### Meeting participants and settings ###

In [10]:
# Local data
data_dir = os.path.join(os.environ.get('HOME'), 'data')
link = 'https://dsets.s3.amazonaws.com/ccb_names_list.csv'
team = FileOP().download_from_url(url=link, download_dir=data_dir)
tm_raw = pd.read_csv(ccb_names_file)
display(tm_raw)

ccb_names_list.csv: 8.19kB [00:00, 29.5kB/s]

Created .csv file.


,name,email,group
0,Andreas,andreas_werdich@hms.harvard.edu,ai-data
1,Gerald,gerald_kiwanuka@hms.harvard.edu,ai-data
2,Nathan,nathan_palmer@hms.harvard.edu,ai-data
3,Sam,samantha_pullman@hms.harvard.edu,ai-data
4,Sreya,sreya_banerjee@hms.harvard.edu,ai-data
5,Andrew,andrew_ghazi@hms.harvard.edu,comp-bio
6,Anthony,anthony-alexander_christidis@hms.harvard.edu,comp-bio
7,Grey,grey_kuling@hms.harvard.edu,comp-bio
8,Ludwig,ludwig_geistlinger@hms.harvard.edu,comp-bio
9,Tram,tram_nguyen@hms.harvard.edu,comp-bio


### Cleanup of the spreadsheet ###
Just in case that was not done. To provide a consistent look.

In [11]:
email_col = 'email'
name_col = 'name'
group_col = 'group'

# Lets do some cleanup of the spreadsheet
# small letters for the column names
tm = tm_raw.copy()
tm.columns = [col.lower() for col in tm.columns]

# small letters for emails and group, first letter capitalize for names
tm[email_col] = tm[email_col].str.lower()
tm[group_col] = tm[group_col].str.lower()
tm[name_col] = tm[name_col].str.title()

# Sort by group and then name
tm = tm.sort_values(by=['group', 'name'], ascending=True).reset_index(drop=True)

# Mark names that are not presenting
rm_name = ['nathan', 'ludwig']
rm_name = [nm.title() for nm in rm_name]
tm = tm.assign(presenting=True)
tm.loc[tm['name'].isin(rm_name), 'presenting'] = False

# save the name ist as a csv_file
presenter_list_file_name = f'{os.path.splitext(group_members_file_name)[0]}_list.csv'
presenter_list_file = os.path.join(data_dir, presenter_list_file_name)
tm.to_csv(presenter_list_file, index=False)
display(tm)

,name,email,group,presenting
0,Andreas,andreas_werdich@hms.harvard.edu,ai-data,True
1,Gerald,gerald_kiwanuka@hms.harvard.edu,ai-data,True
2,Nathan,nathan_palmer@hms.harvard.edu,ai-data,False
3,Sam,samantha_pullman@hms.harvard.edu,ai-data,True
4,Sreya,sreya_banerjee@hms.harvard.edu,ai-data,True
5,Andrew,andrew_ghazi@hms.harvard.edu,comp-bio,True
6,Anthony,anthony-alexander_christidis@hms.harvard.edu,comp-bio,True
7,Grey,grey_kuling@hms.harvard.edu,comp-bio,True
8,Ludwig,ludwig_geistlinger@hms.harvard.edu,comp-bio,False
9,Tram,tram_nguyen@hms.harvard.edu,comp-bio,True


In [12]:
# Remove the names that we do not want in the schedule
presenters_df = tm.loc[tm['presenting']==True]
display(presenters_df)

,name,email,group,presenting
0,Andreas,andreas_werdich@hms.harvard.edu,ai-data,True
1,Gerald,gerald_kiwanuka@hms.harvard.edu,ai-data,True
3,Sam,samantha_pullman@hms.harvard.edu,ai-data,True
4,Sreya,sreya_banerjee@hms.harvard.edu,ai-data,True
5,Andrew,andrew_ghazi@hms.harvard.edu,comp-bio,True
6,Anthony,anthony-alexander_christidis@hms.harvard.edu,comp-bio,True
7,Grey,grey_kuling@hms.harvard.edu,comp-bio,True
9,Tram,tram_nguyen@hms.harvard.edu,comp-bio,True
10,Tyrone,tyrone_lee@hms.harvard.edu,comp-bio,True


### Shuffle the data and create a presenter list ###

In [14]:
# Let's do a random shuffling of the data frame
random_state = 984
presenters_df_shuffled = presenters_df.sample(frac=1, random_state=random_state)

# Get the names and the groups from the new data frame
name_list = list(presenters_df['name'].values)
group_list = list(presenters_df['group'].values)

# Instantiate the Meetings class with the list of names and groups from the data frame
meet = Meetings(name_list=name_list, group_list=group_list)

# We create a presenter list by merging the groups so that we have a member from a different group presenting each time
presenter_list = meet.create_name_sequence()
print(presenter_list)
display(meet.name_df)

['Andreas', 'Andrew', 'Gerald', 'Anthony', 'Sam', 'Grey', 'Sreya', 'Tram', 'Tyrone']


,name,group
0,Andreas,ai-data
1,Andrew,comp-bio
2,Gerald,ai-data
3,Anthony,comp-bio
4,Sam,ai-data
5,Grey,comp-bio
6,Sreya,ai-data
7,Tram,comp-bio
8,Tyrone,comp-bio


### Manipulate the sequence of presenters ###

In [16]:
presenter_list = list(meet.name_df['name'].values)
print(presenter_list)

# If we don't like this sequence, we can create a new one
new_presenter_list = ['Gerald', 'Grey', 'Sam', 'Tyrone', 'Sreya', 'Tram', 'Andreas', 'Anthony', 'Andrew']
print(new_presenter_list)

# We can replace the list of presenters with our new one. Just make sure that the names match.
new_presenter_list = meet.create_name_sequence(name_sequence=new_presenter_list, merge_groups=False)
print(new_presenter_list)

# We can see, that the internal data frame with the group associations has changed according to the new list
print()
display(meet.name_df)

['Gerald', 'Grey', 'Sam', 'Tyrone', 'Sreya', 'Tram', 'Andreas', 'Anthony', 'Andrew']
['Gerald', 'Grey', 'Sam', 'Tyrone', 'Sreya', 'Tram', 'Andreas', 'Anthony', 'Andrew']
['Gerald', 'Grey', 'Sam', 'Tyrone', 'Sreya', 'Tram', 'Andreas', 'Anthony', 'Andrew']



,name,group
0,Gerald,ai-data
1,Grey,comp-bio
2,Sam,ai-data
3,Tyrone,comp-bio
4,Sreya,ai-data
5,Tram,comp-bio
6,Andreas,ai-data
7,Anthony,comp-bio
8,Andrew,comp-bio


### Create the meeting schedule ###

In [17]:
start_date = '2024-08-28'
end_date = '2024-12-31'

cal = meet.create_timeboard(start_date=start_date, 
                            end_date=end_date,
                            start_name='Gerald')

skip_date = '2024-08-28'
skip_name = 'Everyone'
skip_comment = 'Repos, best practices discussion'

cal = meet.skip_date(cal_df=cal, 
                     date=skip_date, 
                     comment=skip_comment,
                     name=skip_name)

display(cal)

,date,name,group,holiday,comment
0,2024-08-28,Everyone,ai-data,False,"Repos, best practices discussion"
1,2024-09-04,Gerald,ai-data,False,None
2,2024-09-11,Grey,comp-bio,False,None
3,2024-09-18,Sam,ai-data,False,None
4,2024-09-25,Tyrone,comp-bio,False,None
5,2024-10-02,Sreya,ai-data,False,None
6,2024-10-09,Tram,comp-bio,False,None
7,2024-10-16,Andreas,ai-data,False,None
8,2024-10-23,Anthony,comp-bio,False,None
9,2024-10-30,Andrew,comp-bio,False,None


In [18]:
# Save the presentation schedule
save_cols = ['date', 'name', 'comment']
schedule_name = f'ccb_presentations_2024.csv'
schedule_file = os.path.join(data_dir, schedule_name)
cal[save_cols].to_csv(schedule_file, index=False)
display(cal[save_cols])

,date,name,comment
0,2024-08-28,Everyone,"Repos, best practices discussion"
1,2024-09-04,Gerald,None
2,2024-09-11,Grey,None
3,2024-09-18,Sam,None
4,2024-09-25,Tyrone,None
5,2024-10-02,Sreya,None
6,2024-10-09,Tram,None
7,2024-10-16,Andreas,None
8,2024-10-23,Anthony,None
9,2024-10-30,Andrew,None
